<h1>Exploratory Data Analysis</h1>

We're going to do some data exploration. For that, we'll need some date. You can find a video game dataset, among 200k other public datasets, available to download at <a href="https://www.kaggle.com/datasets/arnabchaki/popular-video-games-1980-2023?resource=download">Kaggle</a>. We've already downloaded it for you and you'll find it, named *games.csv*, in the 07-data-processing folder.

We will need to install some new libraries for this analysis: pandas and seaborn.

In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install numpy

#import the libraries we'll be using

import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from dateutil.parser import parse

<h3>Read the dataset</h3>
Pandas is a library for working with data, you'll encounter it in nearly any python data analysis project. Here we will use the read_csv() function to create a dataframe.

In [ ]:
filepath = "../games.csv"
data = pd.read_csv(filepath,index_col = 0)
# index_col is just indicating that there is a row number stored in the first column

<h3>Preview the data</h3>

You can use the head() function to view the column headers and first few rows of data.

In [ ]:
data.head()

The shape member gives the number of rows and columns in the entire dataframe. The info() function gives us some information about the columns. Here it's not particularly interesting, but we can see, for example that the rating is a numerical value.

In [ ]:
data.shape

In [ ]:
data.info()

<h3>Missing Values</h3>

Notice the column above labeled "Non-Null Count". See how most columns have 1512 non-null values and this is the number of entries in the dataframe. So those columns are not missing any values.

Note how Team and Rating have fewer than 1512 non-null values? We can display this a bit more clearly with a more custom-built table.

In [ ]:
total_null = data.isnull().sum().sort_values(ascending = False)
percent = ((data.isnull().sum()/data.isnull().count())*100).sort_values(ascending = False)
print("Total records = ", data.shape[0])

missing_data = pd.concat([total_null,percent.round(2)],axis=1,keys=['Total Missing','In Percent'])
missing_data

We can manually replace those missing values. This prevents that they will cause problems with our calculations later. You'll learn more about replacing missing values in Machine Learning and Data Analytics next year.

For now, we'll replace a missing rating with the average of all ratings; we can use the mean function to calculate that. And we'll replace missing text from Team and Summary by "Unknown".

In [ ]:
mean_rating = data['Rating'].mean()

data['Rating'] = data['Rating'].replace(np.nan, mean_rating)
data['Team'] = data['Team'].replace(np.nan, "['Unknown Team']")
data['Summary'] = data['Summary'].replace(np.nan, 'Unknown Summary')

Let's recreate our table from above to check our replacements.

In [ ]:
total_null = data.isnull().sum().sort_values(ascending = False)
percent = ((data.isnull().sum()/data.isnull().count())*100).sort_values(ascending = False)
print("Total records = ", data.shape[0])

missing_data = pd.concat([total_null,percent.round(2)],axis=1,keys=['Total Missing','In Percent'])
missing_data

We've successfully replaced all the missing values.

<h3>Finding Duplicates</h3>

There are also functions for finding and removing duplicate data.

In [ ]:
duplicates = data[data.duplicated()]

duplicates.head()

We're a bit lucky here, because some of the duplicated rows are also present in the head() we saw at the beginning. So we can compare easily and see that Elden Ring at line 0 and line 326 is indeed the same data. Since we're going to drop these from the original dataframe, we can store them here in their own dataframe. IN reality, this is likely unnecessary since we don't have any need to work further with the duplicated rows.

In [ ]:
data = data.drop_duplicates()

data.shape

Now we're down to 1130 rows instead of the original 1512.

Next we'd like to represent the Release Date as a DateTime instead of a string. This allows us to order things chronologically for example.

In [ ]:
# convert the date column to a datetime object
data['Release Date'] = pd.to_datetime(data['Release Date'])

# get the day from the date column
data['Day'] = data['Release Date'].dt.day
data['Month'] = data['Release Date'].dt.strftime('%b')
data['Year'] = data['Release Date'].dt.year
data['Week day'] = data['Release Date'].dt.day_name()

Oops! Something went wrong. See the ValueError at the bottom of the output.

"ValueError: time data "releases on TBD" doesn't match format "%b %d, %Y", at position 423."

So there are some entries in our dataframe that have not (or had not at the time this dataset was scraped) been released. We obviously can't parse a DateTime object out of "releases on TBD".

<h3>Cleaning the Data</h3>

Just like we replaced the missing values (nan) above, we can also replace these TBD dates. For simplicity, we can simply set them to today's date (or some future date if you prefer).

In [ ]:
# create a datetime object with the current time
dt = datetime.now()
# convert the datetime object to a string
dt_str = dt.strftime('%b %d, %Y')
# replace the TBD dates to the current date string
data['Release Date'] = data['Release Date'].str.replace('releases on TBD', dt_str )

Let's try that again.

In [ ]:
# convert the date column to a datetime object
data['Release Date'] = pd.to_datetime(data['Release Date'])

# add columns for Day, Month, and Year based on the Release Date Column data
data['Day'] = data['Release Date'].dt.day
data['Month'] = data['Release Date'].dt.strftime('%b')
data['Year'] = data['Release Date'].dt.year

# add a Week Day column with the day of the week that the game was released on
data['Week day'] = data['Release Date'].dt.day_name()

Let's have another look at the data:

In [ ]:
data.head()

If you recall, Rating is already a numeric value, but Times Listed, Number of Reviews, and others are not. Since we'd like to be able to analyze these, we can also covert these to numeric datatypes.

In [ ]:
# K represents 1000 so we multiply by 1000 in all these columns

data['Times Listed'] = data['Times Listed'].str.replace('K', '').astype(float) * 1000
data['Number of Reviews'] = data['Number of Reviews'].str.replace('K', '').astype(float) * 1000
data['Plays'] = data['Plays'].str.replace('K', '').astype(float) * 1000
data['Playing'] = data['Playing'].str.replace('K', '').astype(float) * 1000
data['Backlogs'] = data['Backlogs'].str.replace('K', '').astype(float) * 1000
data['Wishlist'] = data['Wishlist'].str.replace('K', '').astype(float) * 1000

data.head()

<h3>Descriptive Statistics</h3>

Now that we have converted our strings to numeric values, we can easily compute some descriptive statistics. In fact, pandas provides a describe() function for this.

In [ ]:
data.describe()

<h3>Text Variables</h3>

Have a look again at the data.head() output from earlier. Notice how Team, Genres, and Reviews can have multiple values for a single entry. These columns are actually stored as lists so more than one Team or more than one Genre can be attached to the same game.

Pandas has an explode() function to create separate rows (each with the same index) to represent the multiple values.

In [ ]:
data['Team'] = data['Team'].apply(lambda x: ast.literal_eval(x))

# create a sample DataFrame with a column containing multiple values
df_team = pd.DataFrame({
    'Title': data['Title'].tolist(),
    'Team': data['Team'].tolist()
})
# use the explode method to transform the 'Team' column
df_team = df_team.explode('Team')
df_team

In [ ]:
data['Genres'] = data['Genres'].apply(lambda x: ast.literal_eval(x))

# create a sample DataFrame with a column containing multiple values
df_genres = pd.DataFrame({
    'Title': data['Title'].tolist(),
    'Genres': data['Genres'].tolist()
})
# use the explode method to transform the 'Team' column
df_genres = df_genres.explode('Genres')
df_genres

<h3>Data Visualization</h3>

A Histogram, also called a Distribution graph, shows how many entries (or rows) have a certain value or range of values.

In [ ]:
top_rating = data[['Title','Rating']].sort_values(by = 'Rating', ascending = False)

sns.histplot(data = data['Rating'])

plt.show()


Based on the histogram generated above, what are some conclusions you can make about video game ratings?

Things to consider:
* Are video game ratings evenly spread across the possible ratings or seem to follow a pattern?
* Video games tend to be rated around what value?


Remember how we used explode() to break out the multiple genres for each game. We can use that to show how many of each genre there are. Then plot them in a bar graph or pie chart.

In [ ]:
list_genres = pd.DataFrame(df_genres['Genres'])
count_genres = list_genres.value_counts().plot(kind='bar')

In [ ]:
count_genres = list_genres.value_counts().plot(kind='pie')

The labels get a bit messy here. We can make the chart bigger and move the labels to a legend or leave off some of the less popular genres. Which do you prefer?

In [ ]:
genre_counts = pd.DataFrame(list_genres.value_counts().rename_axis('unique_values'))
genre_counts

#plot = genre_counts.plot(kind = 'pie', labels = None, y='count', figsize=(10, 10))

In [ ]:
shortlist_genres = pd.DataFrame(list_genres.value_counts().head(10))
shortlist_plot = list_genres.value_counts().head(10).plot(kind='pie')

It's worth mentioning, there are criticsms about pie charts in general. Humans aren't very good at determining angles and comparing pie slices. For more discussion on this topic, you can read <a href="https://www.data-to-viz.com/caveat/pie.html">The Issue With the Pie Chart</a>.

<h3>Bivariate Analysis or Correlations</h3>

So far, we've focused on univariate analysis, that is, looking at one variable or column at a time. How many of each genre? What's the average rating? etc.

The next step is to look at how two variables seem to be related to each other. Which genres are played by more players? Is there a relationship between the number of reviews and the average rating?

In [ ]:
shortlist_genres

In [ ]:
# create a sample DataFrame with a column containing multiple values
plays_genre_df = pd.DataFrame({
    'Title': data['Title'].tolist(),
    'Genres': data['Genres'].tolist(),
    'Plays': data['Plays'].tolist(),
    'Playing': data['Playing'].tolist()
})
# use the explode method to transform the 'Genres' column
plays_genre_df = plays_genre_df.explode('Genres')

top10genres = ['Adventure','RPG','Shooter','Platform','Indie','Puzzle','Strategy','Brawler','Simulator','Turn Based Strategy']

plays_genre_df = plays_genre_df.groupby('Genres')[['Plays', 'Playing']].sum().reset_index()
plays_genre_df = plays_genre_df.loc[plays_genre_df['Genres'].isin(top10genres)]
plays_genre_df

In [ ]:
index = plays_genre_df['Genres'].tolist()
value1 = plays_genre_df['Plays'].tolist()
value2 = plays_genre_df['Playing'].tolist()

# Create a horizontal stacked bar chart
fig, ax = plt.subplots(figsize=(8, 6))
bar_width = 0.35
bar1 = ax.barh(index, value1, bar_width, label='Plays', color='#4c72b0')
bar2 = ax.barh(index, value2, bar_width, left=value1, label='Playing', color='#4cbdc9')

# Add labels and legend
ax.set_xlabel('Number of players')
ax.set_ylabel('Genres')
ax.set_title('Number of players for each genre', fontweight='bold')
ax.legend(loc='upper right')


plt.show()

<h2>Task</h2>
Choose a dataset on a topic that's interesting to you. You can search on <a href="https://www.kaggle.com/datasets">Kaggle</a> or another public data repository. Create an Exploratory Data Analysis (what we've done here) of your own.